# LangGraph Chatbot Hands On without LLM

In [1]:
from typing import Annotated, List
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage, AIMessage, BaseMessage
import operator
import gradio as gr
import random
from typing import TypedDict

/Users/ravianand/Documents/VSCODE_PY/MCP_A2A/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Define the State properly using TypedDict
class State(TypedDict):
    messages: Annotated[List[BaseMessage], add_messages]

In [8]:
# Create Nodes that return proper state updates
def greeter(state: State):
    """Welcome message - only sent once per conversation"""
    if not any(isinstance(msg, AIMessage) for msg in state["messages"]):
        return {"messages": [AIMessage(content="👋 Hello! I'm your chatbot.")]}
    return {"messages": []}



In [3]:
def echo_user(state: State):
    """Echo user input"""
    # Get the last user message
    user_messages = [msg for msg in state["messages"] if isinstance(msg, HumanMessage)]
    if user_messages:
        last_user_msg = user_messages[-1]
        return {"messages": [AIMessage(content=f"🗣 You said: {last_user_msg.content}")]}
    return {"messages": []}

In [4]:
def random_fact(state: State):
    """Provide a random fact"""
    facts = [
        "Bananas are berries, but strawberries aren't.",
        "Octopuses have three hearts.",
        "Penguins propose with pebbles."
    ]
    return {"messages": [AIMessage(content=f"🤯 Fun Fact: {random.choice(facts)}")]}


In [5]:
def farewell(state: State):
    """Say goodbye"""
    return {"messages": [AIMessage(content="👋 Goodbye! Come chat again.")]}


In [6]:
# Router function for conditional edges
def router(state: State):
    """Determine which node to execute based on user input"""

    # Get the last user message
    user_messages = [msg for msg in state["messages"] if isinstance(msg, HumanMessage)]
    if not user_messages:
        return "greeter"

    last_user_msg = user_messages[-1]
    user_input = last_user_msg.content.lower()

    if "fact" in user_input:
        return "fact"
    elif any(word in user_input for word in ["bye", "goodbye", "exit", "quit"]):
        return "farewell"
    else:
        return "echo"

In [9]:
# Build Graph
builder = StateGraph(State)

# Add nodes
builder.add_node("greeter", greeter)
builder.add_node("echo", echo_user)
builder.add_node("fact", random_fact)
builder.add_node("farewell", farewell)

# Set up conditional routing
builder.add_conditional_edges(
    START, 
    router, 
    ["greeter", "echo", "fact", "farewell"]
)

In [10]:
# Add edges from nodes to END
builder.add_edge("greeter", END)
builder.add_edge("echo", END)
builder.add_edge("fact", END)
builder.add_edge("farewell", END)

# Compile the graph
graph = builder.compile()


In [11]:
# Helper function to print state in a readable format
def print_state(state, title="State"):
    """Print the state in a readable format"""
    print(f"\n=== {title} ===")
    if "messages" in state:
        print("Messages:")
        for i, msg in enumerate(state["messages"]):
            msg_type = "Human" if isinstance(msg, HumanMessage) else "AI"
            print(f"  {i}: [{msg_type}] {msg.content}")
    else:
        print("No messages in state")
    print("=================\n")


In [12]:
# Create Gradio Chat Interface
def chat_fn(message, chat_history):
    """Handle chat interactions"""
    # Convert chat history to messages
    messages = []
    for user_msg, bot_msg in chat_history:
        messages.append(HumanMessage(content=user_msg))
        messages.append(AIMessage(content=bot_msg))

    # Add the current user message
    messages.append(HumanMessage(content=message))

    # Create initial state
    initial_state = {"messages": messages}

    # Print initial state
    print_state(initial_state, "Initial State")

    try:
        # Execute the graph
        result = graph.invoke(initial_state)

        # Print final state
        print_state(result, "Final State")

        # Extract the new AI responses (only the ones added in this invocation)
        new_ai_messages = [
            msg for msg in result["messages"] 
            if isinstance(msg, AIMessage) and msg not in messages
        ]

        # Get the latest AI response
        latest_response = new_ai_messages[-1].content if new_ai_messages else "I didn't understand that."

        # Append to chat history
        chat_history.append((message, latest_response))
        return "", chat_history

    except Exception as e:
        print(f"Error: {e}")
        chat_history.append((message, "Sorry, I encountered an error. Please try again."))
        return "", chat_history

In [15]:
# Create the interface
with gr.Blocks() as demo:
    gr.Markdown("# 🤖 LangGraph Chatbot Demo")

    chatbot = gr.Chatbot(height=200, label="Conversation")
    msg = gr.Textbox(label="Your Message", placeholder="Type your message here...")
    clear = gr.Button("Clear Chat")

    msg.submit(chat_fn, [msg, chatbot], [msg, chatbot])
    clear.click(lambda: None, None, chatbot, queue=False)

if __name__ == "__main__":
    demo.launch()

* Running on local URL:  http://127.0.0.1:7867
* To create a public link, set `share=True` in `launch()`.
